# Etapa 3,  Preparación de los datos

## Imports

In [1]:
#!pip install imblearn

In [2]:
!pip install imblearn==0.0

^C
ERROR: Operation cancelled by user
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from math import log
sns.set()

from imblearn.under_sampling import RandomUnderSampler

## Carga datos

In [4]:
df=pd.read_csv("./dataset.csv")


KeyboardInterrupt



In [5]:
df.columns

In [63]:
df.head(5)

## Limpieza General

> Eliminamos descriptores no relevantes para la predicción

- Elimino

In [7]:
df.drop(['Chemical formula', 'In literature'], axis = 1, inplace= True)

- Compruebo

In [8]:
df.columns

> Eliminación de registros duplicados

- Comprobamos la cantidad de registros previa

In [9]:
df.shape

- Eliminamos duplicados

In [10]:
df.drop_duplicates(inplace = True)

- Verifcamos si habían duplicados (si algo se elimno

In [11]:
df.shape

> Identificación descriptores con nulos

- Identifico descriptores con nulos

In [12]:
features_presenting_nulls = []

features = df.columns

for feature in features:

    #evaluo la condicion sobre cada descriptor
    condtion = (df[feature] == '-')

    #valido si el descriptor tiene algún true (si hay nulos)
    if (condtion.nunique() > 1):
        features_presenting_nulls.append(feature)
    else:
        continue

- Verifico qué descriptores presentan nulos

In [13]:
features_presenting_nulls

- Convierto '-' en nulls/nans

In [14]:
cat_features = ['Lowest distortion']

for feature in features_presenting_nulls:

    #evaluo la condicion sobre ese descriptor al dataset
    condition = (df[feature]=='-')

    # filtro todos las instancias de esa feature que cumplan esa condicion y los hago None o Nan
    if feature in cat_features:
        df[feature][condition] = None
    else:
        df[feature][condition] = np.nan

- Verifico nulos

In [15]:
df.isnull().sum()

> Eliminamos nulos para descriptores no problemáticos (pocos nulos)

- Aparto descriptores problematicos (m'as nulos), de los que tienen menos

In [16]:
#saco problmeaticos
problematic_features = ['Magnetic moment [mu_B]','Vacancy energy [eV/O atom]']

#saco no problemáticos
no_problematic_features = []
for feature in features_presenting_nulls:
    if feature in problematic_features:
        continue
    else:
        no_problematic_features.append(feature)

- Verificamos si los descpritores menos problemáticos presentan sus nulos en los mismos registros

In [17]:
df_features = []

# creo un df de cada feature con nulos
for feature in no_problematic_features:
    condition = (df[feature].isnull())
    df_features.append(df[feature][condition])

# hago un join (outter) entre todos estos feature con nulos
outer_join = pd.concat(df_features, join='outer', axis=1)

#veo el número de elementos
outer_join.shape

- Eliminamos nulos en descriptores menos problemáticos

In [18]:
#Eliminando instancias con nulos en una de estas columnas, así las elimino en todas
#Me evito usar dropna, porque así eliminaría también los nulos en descriptores problemáticos

#obtengo indices con nulos
lost_data_indexes = []

for index, row in df.iterrows():
    if (row['Lowest distortion'] == None):
        lost_data_indexes.append(index)
    else:
        continue

# elimino instancias con esos indices
df.drop(lost_data_indexes, axis=0, inplace=True)

- Verificamos eliminaciones y nulos

In [19]:
df.info()

## Limpieza Descriptores Numéricos

- conviertir datos categoricos a numericos

In [20]:
df['Formation energy [eV/atom]'] = pd.to_numeric(df['Formation energy [eV/atom]'], errors = 'coerce')
df['Stability [eV/atom]'] = pd.to_numeric(df['Stability [eV/atom]'], errors = 'coerce')
df['Volume per atom [A^3/atom]'] = pd.to_numeric(df['Volume per atom [A^3/atom]'], errors = 'coerce')
df['Magnetic moment [mu_B]'] = pd.to_numeric(df['Magnetic moment [mu_B]'], errors = 'coerce')
df['Band gap [eV]'] = pd.to_numeric(df['Band gap [eV]'], errors = 'coerce')
df['a [ang]'] = pd.to_numeric(df['a [ang]'], errors = 'coerce')
df['b [ang]'] = pd.to_numeric(df['b [ang]'], errors = 'coerce')
df['c [ang]'] = pd.to_numeric(df['c [ang]'], errors = 'coerce')
df['alpha [deg]'] = pd.to_numeric(df['alpha [deg]'], errors = 'coerce')
df['beta [deg]'] = pd.to_numeric(df['beta [deg]'], errors = 'coerce')
df['gamma [deg]'] = pd.to_numeric(df['gamma [deg]'], errors = 'coerce')
df['Vacancy energy [eV/O atom]'] = pd.to_numeric(df['Vacancy energy [eV/O atom]'], errors = 'coerce')


compruebo

In [21]:
df.info()

- Verifico nulos

In [22]:
df.isnull().sum()

In [23]:
df_featuresn = []

# creo un df de cada feature con nulos
for feature in problematic_features:
    condition = (df[feature].isnull()) #todos los nulos que tiene el feature
    df_featuresn.append(df[feature][condition])

# hago un join (inner) entre todos estos feature con nulos
outer_join = pd.concat(df_featuresn, join='inner', axis=1) # 

#veo el número de elementos
outer_join.shape


In [24]:
#obtengo indices con nulos
indexes = outer_join.index
indexes

In [25]:
# elimino instancias con esos indices
df.drop(indexes, axis=0, inplace=True)

In [26]:
df.isna().sum()

In [27]:
df.info()

- Analisis Estadistica , descriptiva y de forma.

In [28]:
df.select_dtypes("number")

In [29]:
df.describe()

- Histograma

In [30]:
fig, ax = plt.subplots(7,2,figsize=(25,55))
columnas=df.select_dtypes("number").columns
cont=0
for i in range(7):
    for j in range(2):
        col=columnas[cont]
        ax[i,j].hist(x=df[col], color='red', rwidth=0.8)
        ax[i,j].set_title(f'Frecuencia de {col}')
        cont+=1
plt.show()


elimino variabeles que tienen el mismo valor

In [31]:
df.drop(['alpha [deg]', 'beta [deg]', 'gamma [deg]'], axis=1)


- Diagrama de bigote

In [32]:
df.info()

In [33]:
#normalize 
df['Stability (log[eV/atom])'] = np.log2(df['Stability [eV/atom]'])
df['Vacancy energy (log[eV/O atom])'] = np.log2(df['Vacancy energy [eV/O atom]'])
df['Magnetic moment (log[mu_B])'] = np.log2(df['Magnetic moment [mu_B]'])

df['Magnetic moment (^2[mu_B])'] = df['Magnetic moment [mu_B]'] ** 2

In [34]:
df_copy = df.copy()

In [35]:
df_copy.dropna(inplace=True)
df_copy.info()

In [36]:
fig, ax = plt.subplots(9,2,figsize=(19,55))
columna=df_copy.select_dtypes("number").columns
cont=0
for i in range(9):
    for j in range(2):

        col=columna[cont]
        ax[i,j].boxplot(x=df_copy[col],vert=True)
        ax[i,j].set_title(f'Diagrama de caja de {col}')
        cont+=1
plt.show()



## Limpieza Descriptores Categóricos

> Normalizar variables categóricas 

- Comprobar si ciertas categorías de cada variable categórica están normalizadas

In [37]:
df['Valence A'].unique()

In [38]:
df['Valence B'].unique()

In [39]:
df['A'].unique()

In [40]:
df['B'].unique()

In [41]:
df['Lowest distortion'].unique()

> Codificación binaria, Target (categórico a numérico)

- Saco lista con labelsc categóricos

In [42]:
categoricalTarget = list(np.unique(df['Lowest distortion']))
categoricalTarget

- Creo lista con labels numéricos

In [43]:
numericalTarget = list(range(0,4))
numericalTarget

- Reemplazo labels categóricos con numéricos

In [44]:
y_cat= df['Lowest distortion'].replace(categoricalTarget, numericalTarget, inplace = False)
list(np.unique(y_cat))

> Codificación binaria, descriptores (categóricos a numéricos

- Extraigo A y B

In [45]:
A = df['A']
B = df['B']

- Saco labels categóricos, del target, A y B, y valido la cantidad de categorías

In [46]:
a_cats = list(np.unique(A))
len(a_cats)

In [47]:
b_cats = list(np.unique(B))
len(b_cats)

- Creo lista con labels numéricos

In [48]:
numerical = list(range(1,len(a_cats)+1))

- Reemplazo labels categóricos con numéricos

In [49]:
A.replace(a_cats, numerical[0:len(a_cats)], inplace = True)
np.unique(A)

In [50]:
B.replace(b_cats, numerical[0:len(b_cats)], inplace = True)
np.unique(B)

> Preparación de la data para OHE

- Saco del dataset aqueelos features categóricos que no voy a dummificar

In [51]:
df.drop(['A', 'B', 'Lowest distortion'], axis=1, inplace = True)

- Valido

In [52]:
df.columns

- Listo las posibles valencias

In [53]:
np.unique(df['Valence A'])

- Observo la cantidad de instancias con valencias ''problemáticas"

In [54]:
df

> OHE para valencias

- Saco los dummies

In [55]:
df_dummies = pd.get_dummies(df)

- Verifico los dummies

In [56]:
df_dummies.columns

## Exploración

- Agrupo todos los datos en un mismo dataset

In [57]:
df = pd.concat([y_cat, A, B,df_dummies], axis=1)

### -. Análisis de correlación

> Eliminemos descriptores por Leakage (no vamos a tenerlos en la predición, ni los vamos a predecir)

In [58]:
df.drop(['alpha [deg]', 'beta [deg]', 'gamma [deg]', 'b [ang]', 'c [ang]'], axis =1,inplace=True);

> Mapa de Calor

- Matriz de correlación

In [59]:
corr_M = df.select_dtypes("number").corr()

- Correlación entre todos los descriptores

In [60]:
plt.figure(figsize=(23,19))
sns.heatmap(corr_M , annot=True)
plt.show()

In [61]:
df.columns

- elimino descriptores correlacionados, teniendo encuenta el criterio de eliminacion por nulos.

In [62]:
#Elimino
df.drop(['Valence A_1', 'Valence A_2', 'Valence A_3', 'Valence A_4', 'Valence A_5', 'Valence A_element not in BVt', 'Valence A_not balanced'], axis=1, inplace=True)

#Valido
df.columns


- eliminamos descriptores igualmente por multicolinealidad entre los originales y los creados

In [0]:
#Elimino
#'Stability [eV/atom]', 'Vacancy energy [eV/O atom]'
df.drop([ 'Magnetic moment [mu_B]', 'Magnetic moment (log[mu_B])'], axis=1, inplace=True)

#Valido
df.columns

> Separo descriptores categóricos de numéricos

- Separo numéricos

In [0]:
df.info()

In [64]:
catgorical_cols = [colname for colname in df.columns if df[colname].dtype in ['int64', 'uint8']]
catgorical_cols

> Diagramas de correlación target - descriptores

- Función para automatizar gráficos de correlación

In [65]:
def plot_correlación(row,fil, target, delete):
    corrDF = pd.DataFrame(df.drop(delete, axis=1, inplace= False).corr()[target].sort_values(ascending=True)).reset_index()
    corrDF.drop(corrDF.shape[0]-1, axis=0, inplace=True)

    if ((fil or row) is False):
        corrSplot = sns.barplot(data= corrDF,x='index', y=target)
    else:
        
        corrSplot = sns.barplot(data= corrDF,x='index', y=target, ax= axs[row, fil])

- Aparto targets numéricos

In [66]:
numerical_targets = [ 'a [ang]']

- Barplot correlación con target categórico

In [67]:
plt.figure(figsize=(14,5))
plot_correlación(False,False, 'Lowest distortion', numerical_targets)

#plt.bar_label(corrSplot.containers[0])
plt.xticks(rotation=90)
plt.show()

- Barplot correlación con targets numéricos

In [68]:
plt.figure(figsize=(14,5))

plot_correlación(False,False, 'a [ang]', 'Lowest distortion')

#plt.bar_label(corrSplot.containers[0])
plt.xticks(rotation=90)
plt.show()

In [69]:
fig, axs = plt.subplots(3, 2, constrained_layout=True, figsize=(14,20))

row = 0
col = 0
for feature in numerical_targets:
    delete = numerical_targets.copy()
    delete.append('Lowest distortion')
    delete.remove(feature)
    
    if col == 2:
        row += 1
        col = 0
    else:
        pass

    plot_correlación(row,col, feature, delete)
    plt.sca(axs[row, col])
    plt.xticks(rotation=90)
    col+=1


### -. Diagrama de dispersión 

> Diagramas de dispersión 3D, data no categórica vs target categórico

In [70]:
scatter_cats_todel = catgorical_cols.copy()
scatter_cats_todel.remove('Lowest distortion')
scatter_cats_todel += numerical_targets

In [71]:
sns.pairplot(data=df.drop(scatter_cats_todel, axis=1, inplace=False), hue='Lowest distortion', palette="Spectral" )

# Exportar dataset procesado

In [72]:
df.to_csv("dataset_clean.csv", sep=',')

# Balanceo (Under o Oversampling), sólo se verificó, no se hizo en el dataset real

- Creo instancia del undersampler

In [73]:
undersample = RandomUnderSampler(random_state=42)

- Undersampling

In [74]:
# Separo datos entre target vs otrasFeatures
x = df.drop('Lowest distortion', axis=1)

y = df['Lowest distortion']

In [75]:
x_over, y_over = undersample.fit_resample(x,y)

- validación

In [76]:
balancedSplot = sns.countplot(x = y_over)
plt.bar_label(balancedSplot.containers[0])
plt.xticks(rotation=45)
plt.show()

- Validemos ahora el espacio de estados sin saturación

In [77]:
full_data = pd.concat([y_over, x_over], axis=1)

In [78]:
sns.pairplot(data=full_data.drop(scatter_cats_todel, axis=1, inplace=False), hue='Lowest distortion', palette="Spectral" )

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a7ebb3ed-5c94-423a-a3b7-e9cd0209ff98' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>